# This notebook uses YOLO implemetation of OpenCV

## Objectives-
#### 1. Run the OpenCV version of YOLO after downloading the weights, config and coco.names files from this link - https://pysource.com/2019/06/27/yolo-object-detection-using-opencv-with-python/
#### 2. Pass the custom image to YOLO to see the output with annotation.
#### 4. Plot the image with BB.

In [1]:
import cv2
import numpy as np

net = cv2.dnn.readNet("./yolov3.weights","./yolo_object_detection/yolov3.cfg")
classes = []
with open("./yolo_object_detection/coco.names", "r") as coco:
    classes = [line.strip() for line in coco.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] -1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

## Load the image to perform Object Detection

In [2]:
img = cv2.imread("./yolo_object_detection/room_self.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Blob conversion using DNN

blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)


## Show the information on the screen

In [3]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

## Non-Max Suppression

In [4]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

## Plot the boundind box on the input image


In [ ]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()